In [1]:
import astroprov
from astropy.io import fits
from astropy.io.ascii import SExtractor
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord
import astropy.coordinates as coord

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
import os
from photutils import DAOStarFinder
from photutils import CircularAperture
from photutils import aperture_photometry
#import subproc

import numpy as np
from astropy import wcs
from astropy.io import fits
import sys

from astroquery.simbad import Simbad

import matplotlib.pyplot as plt
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from astropy.table import Table, vstack
from astropy.table import Column

import collections
import subprocess
from astropy.io import ascii
import warnings, sys


In [22]:
def mywarn(message, category, filename, lineno, file="/home/mj1e16/keplerPhotometry/warnings.log"):
    handle = open(file, 'a')
    handle.write(warnings.formatwarning(message, category, filename, lineno))
    handle.close()
    
warnings.showwarning = mywarn
warnings.warn('test warn message')

In [2]:
def alterDefault(defaultDir,valList,attributeList,catName):
    
    with open(defaultDir+'/default.sex','r') as f:
        data  = f.read()
    for x in range(len(valList)):
        nameLoc = data.find(attributeList[x]) + len(attributeList[x])
        endLoc = data[nameLoc:].find('#') + nameLoc
        newData = data[:nameLoc] + ' '+str(valList[x])+' ' + data[endLoc:]
        data = newData
    
    with open(defaultDir+'/default.sex','r') as f:
        endData = f.read()
    cname = 'CATALOG_NAME'
    nameLoc = endData.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' ' +catName+ ' ' + data[endLoc:]
    
    with open(defaultDir+'/default.sex','r') as f:
        endData = f.read()
    cname = 'PARAMETERS_NAME'
    nameLoc = endData.find(cname) + len(cname)
    endLoc = data[nameLoc:].find('#') + nameLoc
    newData = data[:nameLoc] + ' autodefault.param ' + data[endLoc:]
    
    #print(newData)
    with open(defaultDir+'/default.sex','w') as f:
        f.write(newData)

In [3]:
def sextractor(sextractoryDir,imagename,cataloguename,imageDir='/home/mj1e16/keplerImages/'):
    os.chdir(sextractoryDir)
    subprocess.call(['sex',imageDir+imagename])
    assoc = Table.read(cataloguename,format='ascii.sextractor')

#     os.chdir(imageDir)
#     hdu_list = fits.open(imagename) # open image header
#     imagedata = (hdu_list[0].data) 
    
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (assoc['X_IMAGE'], assoc['Y_IMAGE'])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
#     print(len(assoc))
#     mags = assoc['MAG_APER']
#     plt.hist(mags,bins='auto')
#     plt.show()
    return assoc

In [4]:
def makeDS9RegFile(sexTabList,fileNameBase,tabType):
    for tables in range(len(sexTabList)):
        if tabType == 'dao':
            xcoords = sexTabList[tables]['xcentroid']
            ycoords = sexTabList[tables]['ycentroid']
        elif tabType == 'sex':
            xcoords = sexTabList[tables]['X_IMAGE']
            ycoords = sexTabList[tables]['Y_IMAGE']
        elif tabType == 'iraf':
            xcoords = sexTabList[tables]['X_POS']
            ycoords = sexTabList[tables]['Y_POS']        
        else:
            print('tab type error')
            break
        bigString = 'global color=lightgreen\nimage\n'
        for x in range(len(xcoords)):
            bigString += 'circle({},{},5)\n'.format(xcoords[x],ycoords[x])
        fileName = fileNameBase + '.reg'
        with open(fileName,'w') as f:
            f.write(bigString)
#         print(fileName)
#         print(os.listdir('.'))

In [5]:
def load_wcs_from_file(filename,pixCoord):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[0].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    #print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here.
    # Note also that the pixel coordinates are pairs of [X, Y], 
    # and since WCS built from FITS header automatically has
    # origin set to 1, [0, 0] is actually not inside the image.
    pixcrd = np.array(pixCoord, dtype=np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin" -- in this case we're declaring we
    # have 1-based (Fortran-like) coordinates.
    world = w.wcs_pix2world(pixcrd, 1)
    #print(world)
    return(world)

In [6]:
def queryObjectRegion(outputTable,imageName,inCat,searchRadius=5*u.arcsecond,imDir='/home/mj1e16/simages/'):
    imageCoords = []
    r = searchRadius
    
    if inCat == 'dao':
        xlabel = 'xcentroid'
        ylabel = 'ycentroid'
        label = 'id'
        
    if inCat == 'sex':
        xlabel = 'X_IMAGE'
        ylabel = 'Y_IMAGE'
        label = 'NUMBER'
    flux = []   
    tableLength = len(outputTable)
    xpos = outputTable[xlabel]
    ypos = outputTable[ylabel]
    for x in range(tableLength):
        xpos = outputTable[xlabel][x]
        ypos = outputTable[ylabel][x]
        flux.append(outputTable['FLUX_MAX'][x])
        imageCoords.append([xpos,ypos])
                
    coords = load_wcs_from_file(imDir+imageName,imageCoords)
    customSimbad = Simbad()
    customSimbad.add_votable_fields('ra(d)','dec(d)','otype')
    customSimbad.remove_votable_fields('coordinates')
            
    result_table = Table([['-'],[0],[0],['-'],[0],[0],[0],['-']],names=('MAIN_ID','RA_d','DEC_d','OTYPE','xcentroid','ycentroid','flux','IMAGE'),dtype=('object','float64','float64','object','float64','float64','float64','object'))

    # change to update table and query in one go for the speed
#     for x in range(len(imageCoords)):
#         try:
#             c = SkyCoord(coords[x][0],coords[x][1],frame='icrs',unit='deg')
#             rt = customSimbad.query_region(c,radius=r)
#             rowlist = []
#             for y in range(len(rt[0])):
#                 rowlist.append(rt[0][y])
#             rowlist.append(imageCoords[x][0])
#             rowlist.append(imageCoords[x][1])
#             rowlist.append(flux[x])
#             rowlist.append(imageName)
#             result_table.add_row(rowlist)

#         except:
#             result_table.add_row(['N/A',[0],[0],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],flux[x],imageName])

   #result_table = Simbad.query_region(coord.SkyCoord
    for x in range(len(imageCoords)):
        try:
            c = SkyCoord(coords[x][0],coords[x][1],frame='icrs',unit='deg')
            rt = customSimbad.query_region(c,radius=r)
            rowlist = []
            for y in range(len(rt[0])):
                rowlist.append(rt[0][y])
            rowlist.append(imageCoords[x][0])
            rowlist.append(imageCoords[x][1])
            rowlist.append(flux[x])
            rowlist.append(imageName)
            result_table.add_row(rowlist)

        except:
            result_table.add_row(['N/A',[0],[0],'Object Not Found',[imageCoords[x][0]],[imageCoords[x][1]],flux[x],imageName])

#     hdu_list = fits.open(imageName)
#     imagedata = (hdu_list[0].data) 
#     fig, ax = plt.subplots(figsize=(10, 10))
#     positions = (outputTable[xlabel], outputTable[ylabel])
#     apertures = CircularAperture(positions, r=4.)
#     norm = ImageNormalize(stretch=SqrtStretch())
#     ax.imshow(imagedata, cmap='Greys', origin='lower', vmin=-100,vmax=100)#norm=norm)
#     apertures.plot(color='blue', lw=1.5, alpha=0.5)
#     #plt.savefig('fits.png',dpi=100)
#     plt.show()
    
    return result_table

In [7]:
def runAndShow(image,cat,settings=[10,5,64,3],imDir='/home/mj1e16/Simages/',sexDir='/home/mj1e16/sextractor/sextractor-master/config/'):
    alterDefault(sexDir,settings,['DETECT_THRESH','DETECT_MINAREA','BACK_SIZE','BACK_FILTERSIZE'],catName)
    tab = sextractor(sexDir,image,cat,imageDir=imDir)
    #print(tab)
    pan = tab.to_pandas()
    pan = pan[pan['MAG_BEST'] != 99.0]
    pan = pan[pan['ELLIPTICITY'] < np.median(pan['ELLIPTICITY'])]
    pan = pan[pan['ELONGATION'] < np.median(pan['ELONGATION'])]

#     pan = pan[pan['X_IMAGE'] < 610.]
#     pan = pan[pan['X_IMAGE'] > 570.]
#     pan = pan[pan['Y_IMAGE'] < 300.]
#     pan = pan[pan['Y_IMAGE'] > 250.]
    tab = Table.from_pandas(pan)
    print(len(tab))
    try:
        searchTab = queryObjectRegion(tab,image,'sex',imDir=imDir)
    except:
        pass
#     makeDS9RegFile([tab],'/home/mj1e16/simages/autosextest','sex')
#     subprocess.call(['ds9',imDir+image,'-regions','/home/mj1e16/simages/autosextest.reg'])
    

In [8]:
sexDir = '/home/mj1e16/sextractor/sextractor-master/config/'
catName = 'testauto.cat'
imdir = '/home/mj1e16/Simages/'

In [25]:
diffDir = '/data/mj1e16/kepler/properDiff/'
dirlist = os.listdir(diffDir)

In [27]:
dirlist[0:10]

['diff_imNo23_channel66_ccd58.fits',
 'diff_imNo15_channel61_ccd45.fits',
 'diff_imNo15_channel63_ccd47.fits',
 'diff_imNo34_channel84_ccd32.fits',
 'diff_imNo39_channel73_ccd13.fits',
 'diff_imNo23_channel33_ccd5.fits',
 'diff_imNo8_channel78_ccd78.fits',
 'diff_imNo44_channel9_ccd69.fits',
 'diff_imNo49_channel4_ccd56.fits',
 'diff_imNo42_channel32_ccd56.fits']

In [26]:
for x in range(10):
    outTab = runAndShow(dirlist[x],catName)

95
95
95
95
95
95
95
95
95
95


In [33]:
outTab

MAIN_ID,RA_d,DEC_d,OTYPE,xcentroid,ycentroid,flux,IMAGE
object,float64,float64,object,float64,float64,float64,object
-,0.0,0.0,-,0.0,0.0,0.0,-
N/A,0.0,0.0,Object Not Found,619.1912,30.9739,421.6763,diff_44_1.fits
KIC 8302265,292.7922420622328,44.2687369308147,RotV*,201.8623,32.9643,338.6074,diff_44_1.fits
2MASS J19311294+4414133,292.803941324614,44.2370309160092,EB*,173.7484,42.0038,201.9969,diff_44_1.fits
N/A,0.0,0.0,Object Not Found,904.1633,57.0829,595.7039,diff_44_1.fits
N/A,0.0,0.0,Object Not Found,673.0723,1027.9756,190.9922,diff_44_1.fits
N/A,0.0,0.0,Object Not Found,13.0718,953.7236,606.1566,diff_44_1.fits
KIC 8231835,291.2542958830003,44.1809015727385,RotV*,664.665,922.1371,540.564,diff_44_1.fits
N/A,0.0,0.0,Object Not Found,268.9898,902.938,549.4985,diff_44_1.fits


In [28]:
outTab

In [11]:
Simbad.ROW_LIMIT = 1

imageCoords = []
inCat = 'dao'
imageName = 'diff_44_1.fits'
if inCat == 'dao':
    xlabel = 'xcentroid'
    ylabel = 'ycentroid'
    label = 'id'

if inCat == 'sex':
    xlabel = 'X_IMAGE'
    ylabel = 'Y_IMAGE'
    label = 'NUMBER'
flux = []   
tableLength = len(outTab)
xpos = outTab[xlabel]
ypos = outTab[ylabel]
flux = outTab['flux']

imageCoords = [[xp,yp] for xp,yp in zip(xpos,ypos)]
coords = load_wcs_from_file(imdir+imageName,imageCoords)
print(len(coords))
RA = [x[0] for x in coords]
DEC= [x[1] for x in coords]

result_table = Simbad.query_region(coord.SkyCoord(ra=RA,dec=DEC,unit=(u.deg,u.deg),frame='fk5'),radius=5*u.arcsec)


95


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): '19:31:59.7575 +44:05:50.7695': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 7 raised an error (recorded in the `errors` attribute of the result table): '19:28:45.2688 +44:54:51.7894': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/astroquery/simbad/core.py:136: UserWarning: Warning: The script line number 9 raised an error (recorded in the `errors` attribute of the result table): '19:26:59.5233 +43:33:14.0408': No astronomical object found :  
  (error.line, error.msg))
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packa

In [14]:
print(RA)

[292.99898943109656, 292.45720774996073, 292.7934534939304, 292.8042407015132, 292.1886198689317, 291.10934790279765, 291.74801359041527, 291.2534826069752, 291.6046318111972, 291.718850407735, 291.2790863500245, 291.0474767648147, 291.3585608922904, 291.3432508469234, 291.1009123624769, 291.8826951000789, 291.3914815159339, 291.5405162290623, 291.4596699011871, 291.5596135485589, 291.88762391766574, 291.509969376059, 292.06616997219714, 291.52579494027464, 292.1593159823193, 291.5992619278863, 292.0286932635259, 292.01405262724217, 291.82943758981924, 291.3559317993328, 291.53013874668534, 291.6439890686545, 291.4110622945645, 292.1533270326032, 292.07605017906326, 291.9007587669276, 292.2306124746715, 291.9475409176377, 292.0459226327159, 292.01055769976756, 291.510967955832, 292.2729076916256, 291.5835477376634, 291.67960235170966, 292.03173457157953, 291.7522175097704, 292.0663569589308, 292.0386308958731, 292.2626389520863, 291.71068764171133, 292.34527034223817, 292.1062980512097

In [12]:
len(result_table)

34

In [13]:
result_table

MAIN_ID,RA,DEC,RA_PREC,DEC_PREC,COO_ERR_MAJA,COO_ERR_MINA,COO_ERR_ANGLE,COO_QUAL,COO_WAVELENGTH,COO_BIBCODE
,"""h:m:s""","""d:m:s""",,,mas,mas,deg,,,
object,unicode13,unicode13,int16,int16,float32,float32,int16,unicode1,unicode1,object
KIC 8302265,19 31 10.1380,+44 16 07.452,14,14,0.019,0.021,90,A,O,2018yCat.1345....0G
2MASS J19311294+4414133,19 31 12.9459,+44 14 13.311,14,14,0.045,0.047,90,A,O,2018yCat.1345....0G
KIC 8231835,19 25 01.0310,+44 10 51.245,14,14,0.015,0.018,90,A,O,2018yCat.1345....0G
KIC 8557981,19 24 11.2090,+44 39 08.636,14,14,0.021,0.022,90,A,O,2018yCat.1345....0G
KIC 8558132,19 24 24.4278,+44 40 51.125,14,14,0.017,0.020,90,A,O,2018yCat.1345....0G
KIC 8028961,19 27 31.7028,+43 48 01.913,14,14,0.014,0.016,90,A,O,2018yCat.1345....0G
KIC 8364437,19 25 34.2344,+44 22 45.694,14,14,0.013,0.015,90,A,O,2018yCat.1345....0G
KIC 7960729,19 28 15.7725,+43 45 06.980,14,14,0.021,0.024,90,A,O,2018yCat.1345....0G


In [12]:
a = [0,1,2]
b = [10,1,2]
x = 0
c = lambda a, b: [a[x],b[x]]
print(c(a,b))

[0, 10]


In [13]:
c = [[val,pair] for val, pair in zip(a,b)]
print(c)

[[0, 10], [1, 1], [2, 2]]


In [10]:
a = lambdafunc(a,b,x)
print(a)

<function <lambda> at 0x7fda24594848>


In [1]:
dirlist = os.listdir(imdir)
dirlist = [x for x in dirlist if x[-4:] =='fits'] # check
#print(dirlist)


for im in range(len(dirlist)):
    if im == 0:
        outTab = runAndShow(dirlist[im],catName,imDir=imdir)
    else:
        newTab = runAndShow(dirlist[im],catName,imDir=imdir)
        try:
            outTab = vstack([outTab,newTab])
        except:
            print('No transients found in Image '+dirlist[im])
df = outTab.to_pandas()
df = df[df['OTYPE'] == 'Object Not Found']
dfsort = df.sort_values(['FLUX_MAX'])

In [69]:
outPan = outTab.to_pandas()
sortTab = outPan.sort_values(['ELONGATION'])


In [29]:
dirlist = os.listdir(imdir)
dirlist = [x for x in dirlist if x[-4:] =='fits'] # check
print(dirlist)

['difkplr2009115080620_ffi-cal57.fits', 'difkplr2009115053616_ffi-cal68.fits', 'difkplr2010140101631_ffi-cal64.fits', 'difkplr2012004204112_ffi-cal15.fits', 'difkplr2012179140901_ffi-cal84.fits', 'difkplr2011177110110_ffi-cal80.fits', 'difkplr2010326181728_ffi-cal50.fits', 'difkplr2010265195356_ffi-cal76.fits', 'difkplr2009322233047_ffi-cal10.fits', 'difkplr2012151105138_ffi-cal45.fits', 'difkplr2012277203051_ffi-cal76.fits', 'difkplr2010078174524_ffi-cal84.fits', 'difkplr2009115173611_ffi-cal24.fits', 'difkplr2010111125026_ffi-cal56.fits', 'difkplr2012179140901_ffi-cal20.fits', 'difkplr2010174164113_ffi-cal30.fits', 'difkplr2009114204835_ffi-cal3.fits', 'difkplr2009292020429_ffi-cal71.fits', 'difkplr2010203012215_ffi-cal34.fits', 'difkplr2012277203051_ffi-cal34.fits', 'difkplr2009170043915_ffi-cal66.fits', 'difkplr2012277203051_ffi-cal16.fits', 'difkplr2011024134926_ffi-cal79.fits', 'difkplr2010203012215_ffi-cal25.fits', 'difkplr2009260000800_ffi-cal45.fits', 'difkplr2010326181728_ffi

In [70]:
sortTab

,NUMBER,X_IMAGE,Y_IMAGE,ALPHA_J2000,DELTA_J2000,FLUX_APER,FLUXERR_APER,MAG_APER,FLUX_RADIUS,FWHM_IMAGE,BACKGROUND,THRESHOLD,FLUX_MAX,ISOAREA_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,FLAGS,ELONGATION,ELLIPTICITY
1626,2889,407.4532,409.4460,0.0,0.0,1227.86600,9.083923,-7.7229,0.804,0.98,-0.669483,6.13679,366.29710,8,0.870,0.869,-70.58,0,1.001,0.001
2449,4183,480.9913,582.9999,0.0,0.0,147.95830,9.065479,-5.4253,0.419,0.00,-0.420949,6.13679,129.85340,3,0.715,0.714,71.92,0,1.002,0.002
3449,5657,636.6011,667.0096,0.0,0.0,927.34060,9.065479,-7.4181,0.722,1.07,-0.422779,6.13679,390.62750,9,0.852,0.850,48.01,0,1.002,0.002
2948,4872,177.2634,779.3145,0.0,0.0,4380.21600,9.037742,-9.1037,0.820,1.69,-0.285206,6.13679,1184.95400,10,0.921,0.919,65.99,0,1.002,0.002
169,296,952.9039,55.9717,0.0,0.0,11128.91000,9.046997,-10.1161,0.956,2.55,-0.037754,6.13679,2427.59500,12,0.973,0.972,-56.56,0,1.002,0.002
3336,5470,415.5953,684.6047,0.0,0.0,731.91220,9.065478,-7.1611,0.788,0.98,-0.372609,6.13679,252.34390,7,0.866,0.865,56.00,0,1.002,0.002
808,1527,277.9213,219.6775,0.0,0.0,2729.95500,9.065479,-8.5904,0.737,1.34,-0.762254,6.13679,1094.57900,8,0.904,0.902,62.21,3,1.002,0.002
1830,3211,141.9317,445.3230,0.0,0.0,2254.89200,9.037742,-8.3828,1.219,2.12,-0.613897,6.13679,785.11540,13,1.150,1.148,71.16,0,1.002,0.002
3319,5445,364.4733,689.9902,0.0,0.0,669.48400,9.074706,-7.0644,0.734,1.19,-0.445358,6.13679,239.15020,7,0.854,0.852,-37.33,0,1.002,0.002
2299,3959,824.2881,548.0385,0.0,0.0,807.67320,9.065479,-7.2681,0.708,1.15,-0.340390,6.13679,358.38990,8,0.851,0.848,-65.54,0,1.003,0.003


In [99]:
print(np.median(outTab['ELLIPTICITY']))

nan


/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/numpy/core/fromnumeric.py:2954: RuntimeWarning: Mean of empty slice.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/home/mj1e16/miniconda2/envs/astroconda/lib/python2.7/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [100]:
print(np.median(outTab['ELONGATION']))
print(np.mean(outTab['ELONGATION']))

nan
nan
